# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
S. Li  ->  S. Li  |  ['S. Li']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 51 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.02726


extracting tarball to tmp_2408.02726...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.03075


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2686/1211882699.py:51: LatexWarning: 2408.02726 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


/tmp/ipykernel_2686/1211882699.py:51: LatexWarning: 2408.03075 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2408.03189


extracting tarball to tmp_2408.03189...

 done.


Found 136 bibliographic references in tmp_2408.03189/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.03189-b31b1b.svg)](https://arxiv.org/abs/2408.03189) | **21 Balmer Jump Street: The Nebular Continuum at High Redshift and Implications for the Bright Galaxy Problem, UV Continuum Slopes, and Early Stellar Populations**  |
|| H. Katz, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-08-07*|
|*Comments*| *26 pages, 18 figures, submitted to The Open Journal of Astrophysics*|
|**Abstract**|            We study, from both a theoretical and observational perspective, the physical origin and spectroscopic impact of extreme nebular emission in high-redshift galaxies. The nebular continuum, which can appear during extreme starbursts, is of particular importance as it tends to redden UV slopes and has a significant contribution to the UV luminosities of galaxies. Furthermore, its shape can be used to infer the gas density and temperature of the ISM. First, we provide a theoretical background, showing how different stellar populations (SPS models, IMFs, and stellar temperatures) and nebular conditions impact observed galaxy spectra. We demonstrate that, for systems with strong nebular continuum emission, 1) UV fluxes can increase by up to 0.7~magnitudes (or more in the case of hot/massive stars) above the stellar continuum, which may help reconcile the surprising abundance of bright high-redshift galaxies and the elevated UV luminosity density at $z>10$, 2) at high gas densities, UV slopes can redden from $\beta\lesssim-2.5$ to $\beta\sim-1$, 3) observational measurements of $\xi_{ion}$ are grossly underestimated, and 4) UV downturns from two-photon emission can masquerade as DLAs. Second, we present a dataset of 58 galaxies observed with NIRSpec on JWST at $2.5<z<9.0$ that are selected to have strong nebular continuum emission via the detection of the Balmer jump. Five of the 58 spectra are consistent with being dominated by nebular emission, exhibiting both a Balmer jump and a UV downturn consistent with two-photon emission. For some galaxies, this may imply the presence of hot massive stars and a top-heavy IMF. We conclude by exploring the properties of spectroscopically confirmed $z>10$ galaxies, finding that UV slopes and UV downturns are in some cases redder or steeper than expected from SPS models, which may hint at more exotic (e.g. hotter/more massive stars or AGN) ionizing sources.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.02726-b31b1b.svg)](https://arxiv.org/abs/2408.02726) | **JADES Ultra-red Flattened Objects: Morphologies and Spatial Gradients in Color and Stellar Populations**  |
|| J. L. Gibson, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-08-07*|
|*Comments*| *24 pages, 13 figures; accepted for publication in The Astrophysical Journal*|
|**Abstract**|            One of the more surprising findings after the first year of JWST observations is the large number of spatially extended galaxies (ultra-red flattened objects, or UFOs) among the optically-faint galaxy population otherwise thought to be compact. Leveraging the depth and survey area of the JADES survey, we extend observations of the optically-faint galaxy population to an additional 112 objects, 56 of which are well-resolved in F444W with effective sizes, $R_e > 0.25''$, more than tripling previous UFO counts. These galaxies have redshifts around $2 < z < 4$, high stellar masses ($\mathrm{log(M_*/M_{\odot})} \sim 10-11$), and star-formation rates around $\sim 100-1000 \mathrm{M_{\odot}/yr}$. Surprisingly, UFOs are red across their entire extents which spatially resolved analysis of their stellar populations shows is due to large values of dust attenuation (typically $A_V > 2$ mag even at large radii). Morphologically, the majority of our UFO sample tends to have low Sérsic indices ($n \sim 1$) suggesting these large, massive, optically faint galaxies have little contribution from a bulge in F444W. Further, a majority have axis-ratios between $0.2 < q < 0.4$, which Bayesian modeling suggests that their intrinsic shapes are consistent with being a mixture of inclined disks and prolate objects with little to no contribution from spheroids. While kinematic constraints will be needed to determine the true intrinsic shapes of UFOs, it is clear that an unexpected population of large, disky or prolate objects contributes significantly to the population of optically faint galaxies.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.03075-b31b1b.svg)](https://arxiv.org/abs/2408.03075) | **Characterizing the current systems in the Martian ionosphere**  |
|| J. Gao, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-08-07*|
|*Comments*| *20 pages, 6 figures*|
|**Abstract**|            When the solar wind interacts with the ionosphere of an unmagnetized planet, it induces currents that form an induced magnetosphere. These currents and their associated magnetic fields play a pivotal role in controlling the movement of charged particles, which is essential for understanding the escape of planetary ions. Unlike the well-documented magnetospheric current systems, the ionospheric current systems on unmagnetized planets remain less understood, which constrains the quantification of electrodynamic energy transfer from stars to these planets. Here, utilizing eight years of data from the Mars Atmosphere and Volatile EvolutioN (MAVEN) mission, we investigate the global distribution of ionospheric currents on Mars. We have identified two distinct current systems in the ionosphere: one aligns with the solar wind electric field yet exhibits hemispheric asymmetry perpendicular to the electric field direction; the other corresponds to the flow pattern of annually-averaged neutral winds. We propose that these two current systems are driven by the solar wind and atmospheric neutral winds, respectively. Our findings reveal that Martian ionospheric dynamics are influenced by the neutral winds from below and the solar wind from above, highlighting the complex and intriguing nature of current systems on unmagnetized planets.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.03189/././figures/all_sps_beta.png', 'tmp_2408.03189/././figures/temp_beta_indiv.png', 'tmp_2408.03189/././figures/neb_only_beta.png', 'tmp_2408.03189/././figures/neb_only_EWs.png', 'tmp_2408.03189/././figures/neb_only_xi_ion.png', 'tmp_2408.03189/././figures/two_phot_galaxy_comparison.png']
copying  tmp_2408.03189/././figures/all_sps_beta.png to _build/html/
copying  tmp_2408.03189/././figures/temp_beta_indiv.png to _build/html/
copying  tmp_2408.03189/././figures/neb_only_beta.png to _build/html/
copying  tmp_2408.03189/././figures/neb_only_EWs.png to _build/html/
copying  tmp_2408.03189/././figures/neb_only_xi_ion.png to _build/html/
copying  tmp_2408.03189/././figures/two_phot_galaxy_comparison.png to _build/html/
exported in  _build/html/2408.03189.md
    + _build/html/tmp_2408.03189/././figures/all_sps_beta.png
    + _build/html/tmp_2408.03189/././figures/temp_beta_indiv.png
    + _build/html/tmp_2408.03189/././figures/neb_only_beta.png
    + _build/html/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\cmark}{\textcolor{green}{\ding{51}}}$
$\newcommand{\ct}{C {\small III}}$
$\newcommand{\fesc}{f_{\rm esc}}$
$\newcommand{\fescm}{f_{\rm esc}}$
$\newcommand{\het}{He {\small II}}$
$\newcommand{\kms}{\mbox{{\rm km s^{-1}}}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\mgt}{Mg {\small II}}$
$\newcommand{\msun}{\mbox{M_{\odot}}}$
$\newcommand{\msunyr}{\mbox{\rm M_{\odot} {\rm yr^{-1}}}}$
$\newcommand{\mstar}{\mbox{M_{*}}}$
$\newcommand{\MUV}{\rm M_{UV}}$
$\newcommand{\mvir}{\mbox{M_{\rm vir}}}$
$\newcommand{\neth}{Ne {\small III}}$
$\newcommand{\ot}{O {\small II}}$
$\newcommand{\oth}{O {\small III}}$
$\newcommand{\SFRten}{\mbox{{\rm SFR}_{\rm 10}}}$
$\newcommand{\xmark}{\textcolor{red}{\ding{55}}}$
$\newcommand{\App}[1]{Appendix~\ref{#1}}$
$\newcommand{\eq}[1]{(\ref{#1})}$
$\newcommand{\Eq}[1]{Eq.~(\ref{#1})}$
$\newcommand{\Eqs}[1]{Eqs.~(\ref{#1})}$
$\newcommand{\Fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\Sec}[1]{\S\ref{#1}}$
$\newcommand{\Tab}[1]{Table~\ref{#1}}$
$\newcommand\Omegab{\Omega_{\rm b}}$
$\newcommand\Omegam{\Omega_{\rm m}}$
$\newcommand\OmegaDM{\Omega_{\rm DM}}$
$\newcommand{\comment}{\textcolor{red}}$</div>



<div id="title">

# 21 Balmer Jump Street: The Nebular Continuum at High Redshift and Implications for the Bright Galaxy Problem, UV Continuum Slopes, and Early Stellar Populations$\vspace{-15mm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.03189-b31b1b.svg)](https://arxiv.org/abs/2408.03189)<mark>Appeared on: 2024-08-07</mark> -  _26 pages, 18 figures, submitted to The Open Journal of Astrophysics_

</div>
<div id="authors">

H. Katz, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We study, from both a theoretical and observational perspective, the physical origin and spectroscopic impact of extreme nebular emission in high-redshift galaxies. The nebular continuum, which can appear during an extreme starburst, is of particular importance as it tends to redden UV slopes and has a significant contribution to the UV luminosities of galaxies. Furthermore, its shape can be used to infer the gas density and temperature of the interstellar medium. First, we provide a theoretical background, showing how different stellar populations (SPS models, initial mass functions (IMFs), and stellar temperatures) and nebular conditions impact observed galaxy spectra. We demonstrate that, for systems with strong nebular continuum emission, 1) UV fluxes can increase by up to 0.7 magnitudes (or more in the case of hot/massive stars) above the stellar continuum, which may help reconcile the surprising abundance of bright high-redshift galaxies and the elevated UV luminosity density at $z\gtrsim10$ , 2) at high gas densities, UV slopes can redden from $\beta\lesssim-2.5$ to $\beta\sim-1$ , 3) observational measurements of $\xi_{\rm ion}$ are grossly underestimated, and 4) UV downturns from two-photon emission can masquerade as damped Ly $\alpha$ systems. Second, we present a dataset of 58 galaxies observed with NIRSpec on JWST at $2.5<z<9.0$ that are selected to have strong nebular continuum emission via the detection of the Balmer jump. Five of the 58 spectra are consistent with being dominated by nebular emission, exhibiting both a Balmer jump and a UV downturn consistent with two-photon emission. For some galaxies, this may imply the presence of hot massive stars and a top-heavy IMF. We conclude by exploring the properties of spectroscopically confirmed $z>10$ galaxies, finding that UV slopes and UV downturns are in some cases redder or steeper than expected from SPS models, which may hint at more exotic (e.g. hotter/more massive stars or AGN) ionizing sources.

</div>

<div id="div_fig1">

<img src="tmp_2408.03189/././figures/all_sps_beta.png" alt="Fig5.1" width="50%"/><img src="tmp_2408.03189/././figures/temp_beta_indiv.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** UV slope as a function of stellar age for various SPS models (top) or stellar temperature for the individual metal-poor star models (bottom). Solid lines represent the UV slope as measured from total (stellar$+$nebular) emission while dashed lines represent the UV slope of only the stellar component. The dotted red lines indicate a slope of $-2.3$ for reference in all panels. For the Starburst99 models, colors represent different upper-mass IMF slopes (see legend in top panel), while for the C\&B19 models, colors indicate different upper-mass limits (see legend in the third panel). For the individual massive star models, colors indicate the gas density of the nebula.  (*fig:sps_beta*)

</div>
<div id="div_fig2">

<img src="tmp_2408.03189/././figures/neb_only_beta.png" alt="Fig11.1" width="33%"/><img src="tmp_2408.03189/././figures/neb_only_EWs.png" alt="Fig11.2" width="33%"/><img src="tmp_2408.03189/././figures/neb_only_xi_ion.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** (Left) UV slope, $\beta$ as a function of gas temperature for the nebular continuum assuming Case B recombination. (Center) Equivalent widths of H$\alpha$ and H$\beta$ as a function of temperature. Solid and dashed lines show the results with and without two-photon emission, respectively. (Right) $\log_{10}(\xi_{\rm ion})$ as a function of gas temperature for the nebular continuum. (*fig:neb_only_beta*)

</div>
<div id="div_fig3">

<img src="tmp_2408.03189/././figures/two_phot_galaxy_comparison.png" alt="Fig17" width="100%"/>

**Figure 17. -** Comparison of 1210\_5217 at $z=4.890$ with potential nebular-dominated galaxies 2189\_7807 ($z=5.387$) and GS\_9422 ($z=5.946$). All three show a clear Balmer jump and a very similar UV downturn. Spectral modeling predicts a very high nebular fraction in each of these objects. (*fig:two_phot_spectra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.03189"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

153  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
